In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
warnings.filterwarnings("ignore")

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://produccion.local/api/user/login/", login, verify=False)
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://produccion.local/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info

In [ ]:
# #CODIGO PARA AÑADIR UN NUEVO PRODUCTOR
# objeto_productor = {
#     "documento": 65733348,
#     "productor_data": {
#         "activo":True,
#         "delegado": False,
#         "nombre_completo": "RAMOS GONZALEZ ANA LOLEINA",
#         "fecha_afiliacion": "2025-11-07",
#         "genero": "F"
#     },
#     "habilidad": {
#         "habil": True
#     }
# }
# response = session.post("https://produccion.local/api/v1/productor/", json=objeto_productor, verify=False)
# print(response.status_code)
# print(response.text)

In [ ]:
file_shp = "~/Descargas/55056272_Lotes 6.geojson"
gdf_adicionar = gpd.read_file(file_shp)
documento_productor = 55056272

In [ ]:
gdf_adicionar = gdf_adicionar.to_crs(3116)
gdf_adicionar["area_individual"] = gdf_adicionar.geometry.area/10000
gdf_adicionar = gdf_adicionar.to_crs(4326)

In [ ]:
gdf_adicionar.columns

In [ ]:
gdf_adicionar["numero_lote"]

In [ ]:
def retornar_numero_lote(valor):
    try:
        lote_nombre = valor.split("0")[1]
        print(lote_nombre)
        lote_numero = lote_nombre.split(" ")[-1]
        return int(lote_numero)
    except (IndexError, ValueError):
        return None  # o np.nan si usas pandas

gdf_adicionar["numero_lote"] = gdf_adicionar['Name'].apply(lambda x: retornar_numero_lote(x))

In [ ]:
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry.base import BaseGeometry

def remove_z(geom: BaseGeometry):
    if geom is None:
        return None
    # POLYGON
    if geom.geom_type == 'Polygon':
        exterior = [(x, y) for x, y, *_ in geom.exterior.coords]
        interiors = [
            [(x, y) for x, y, *_ in ring.coords] for ring in geom.interiors
        ]
        return Polygon(exterior, interiors)

    # MULTIPOLYGON
    elif geom.geom_type == 'MultiPolygon':
        polygons = [remove_z(poly) for poly in geom.geoms]
        return MultiPolygon(polygons)

    # Si ya es 2D o no reconocido, retornar igual
    return geom

# Aplicar al GeoDataFrame
gdf_adicionar["geometry"] = gdf_adicionar["geometry"].apply(remove_z)



In [ ]:
gdf_adicionar = gdf_adicionar.explode(index_parts=False).reset_index(drop=True)
# gdf_adicionar.to_file(file_shp)

In [ ]:
gdf_adicionar.iloc[0].to_dict()

In [ ]:
gdf_adicionar["Estad_Cult"] = gdf_adicionar["Estad_Cult"].apply(
    lambda x: 1 if x == "Produccion" else 0
)
gdf_adicionar["Estad_Cult"].unique()


In [ ]:
gdf_adicionar["Tipo_Activ"].unique()

In [ ]:
gdf_adicionar["Tipo_Activ"] = gdf_adicionar["Tipo_Activ"].apply(
    lambda x: 0 if x == 'NS - NUEVA SIEMBRA' else 2
)


In [ ]:
for i, row in gdf_adicionar.iterrows():
    respuesta = {
        "documento_productor":documento_productor,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "numero_documento": str(documento_productor),
            "fecha_visita": pd.to_datetime(row["Fecha_Vis"]).strftime("%Y-%m-%d"),
            "area": row["area_individual"],
            "observaciones": "",
            "descripcion": row["Descri_Lot"] if not pd.isna(row["Descri_Lot"]) else "",
            "numero_lote": row["Numero_Lot"],
            "variedad": row["C_Variedad"],
            "distancia_surcos": row["Dist_Surco"],
            "distancia_plantas": row["Dist_Plant"],
            "densidad": row["C_Densidad"],
            "numero_plantas": row["N_Plantas"],
            "gramos_plantas": row["gr_Planta"],
            "kg_produccion": row["Prod_Plant"],
            "fecha_actividad": pd.to_datetime(row["Fecha_Acti"]).strftime("%Y-%m-%d"),
            "produccion": row["Estad_Cult"],
            "estado": "BUENO",
            "subtipo_operacion": row["Tipo_Activ"]
        }
    }
    session.post("https://produccion.local/api/v1/poligonos_lotes/", json=respuesta, verify=False)
    print(respuesta)
    print(f"Status: {response.status_code} - {response.text}")

In [ ]:
for i, row in gdf_adicionar.iterrows():
    respuesta = {
        "documento_productor":documento_productor,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "numero_documento": str(documento_productor),
            "fecha_visita": "2025-03-25",
            "area": row["area_individual"],
            "observaciones": "",
            "descripcion": "",
            "numero_lote": 0,
            "variedad": "",
            "distancia_surcos": 150,
            "distancia_plantas": 120,
            "densidad": 5500,
            "numero_plantas": 1,
            "gramos_plantas": 500,
            "kg_produccion": 1,
            "fecha_actividad": "",
            "produccion": 1,
            "estado": "BUENO",
            "subtipo_operacion": 2
        }
    }
    session.post("https://produccion.local/api/v1/poligonos_lotes/", json=respuesta, verify=False)
    print(respuesta)
    print(f"Status: {response.status_code} - {response.text}")

In [ ]:
import json
json = {
    "poligono": str(gdf_adicionar["geometry"].iloc[0])
}
print(json)

session.patch(f"https://produccion.local/api/v1/poligonos_lotes/bf622896-042e-4dfd-b969-eb64af4bcc44/", json=json,verify=False)

In [ ]:
# gdf_adicionar["nombre_fin"]=gdf_adicionar["nombre_fin"].fillna("")
# gdf_adicionar["fecha_visi"]=gdf_adicionar["fecha_visi"].fillna("")

In [ ]:
gdf_adicionar

In [ ]:
gdf_adicionar.columns

In [ ]:
#REPORTE DE FINCAS
for i, row in gdf_adicionar.iterrows():
    # print(str(row["geometry"]))
    respuesta = {
        "documento_productor":documento_productor,
        "poligono":str(row["geometry"]),
        "mongo_atribute":{
            "nombre_finca": "LA FLORIDA",
            "fecha_visita": "2025-03-25"
        }
    }
    response = session.post("https://produccion.local/api/v1/poligonos_fincas/", json=respuesta, verify = False)
    print(respuesta)
    print(f"Status: {response.status_code} - {response.text}")